# Image Gan Notebook
**Jorge H, Hao W, Luke V, Tom N**

w251 - Spring 2021, UC Berkeley

In [ ]:
# For keeping tensorflow version consistent
!pip3 install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 64kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
     |████████████████████████████████| 450kB 40.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=40558294d232a6c440e64fd7b14926de208c9d327d369ff6806d0786edd0a1f5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tenso

In [ ]:
# For a dependency
!pip3 install mido

     |████████████████████████████████| 61kB 6.0MB/s 


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D,ZeroPadding1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv1D,UpSampling1D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
import pandas as pd
from mido import MidiFile, MidiTrack, Message
from music21 import *
from IPython.display import Image

In [ ]:
print(tf.__version__)

2.0.0


In [ ]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

The following block contains parameters used later on in the notebook.

In [ ]:
# Size vector to generate images from
SEED_SIZE = 100

# Configuration
EPOCHS = 50
BATCH_SIZE = 32
BUFFER_SIZE = 60000

# Midi data

This sections is for loading the npy maestro data and perparing it for input into the GAN models.

In [ ]:
!ls

All_Maestro_Parsed.npy	sample_data


In [ ]:
training_data_midi = np.load('All_Maestro_Parsed.npy')

In [ ]:
training_data_midi.shape

(15300, 200, 3)

In [ ]:
# training_data_midi[:,:,1] = training_data_midi[:,:,1]*100
# training_data_midi[:,:,2] = training_data_midi[:,:,2]*100

In [ ]:
training_data_midi[0]

array([[1.81818182e-01, 8.45376135e-03, 0.00000000e+00],
       [2.61363636e-01, 9.27628948e-02, 1.71360027e-04],
       [1.70454545e-01, 1.23379220e-02, 7.19712115e-03],
       [1.25000000e-01, 5.94048095e-02, 7.19712115e-03],
       [1.81818182e-01, 6.07756897e-02, 5.27217684e-02],
       [2.72727273e-01, 2.09630434e-02, 1.73073628e-02],
       [2.50000000e-01, 2.23339236e-02, 1.57651225e-02],
       [2.61363636e-01, 9.82464157e-02, 1.97635232e-02],
       [1.70454545e-01, 7.59696122e-03, 7.31136117e-03],
       [1.25000000e-01, 5.85480094e-02, 8.22528132e-03],
       [1.81818182e-01, 6.32318501e-02, 5.12937682e-02],
       [2.72727273e-01, 2.51328040e-02, 2.16484835e-02],
       [2.50000000e-01, 1.81641629e-02, 1.79356829e-02],
       [2.61363636e-01, 2.62809162e-01, 1.58793625e-02],
       [1.70454545e-01, 1.02244816e-02, 7.31136117e-03],
       [1.25000000e-01, 3.90129662e-02, 8.79648141e-03],
       [1.81818182e-01, 2.05632033e-03, 4.03838465e-02],
       [1.81818182e-01, 4.16976

In [ ]:
# Smaller testing section of the training data
train_data_midi_temp = training_data_midi[:3000,:192,:]

In [ ]:
train_data_midi_temp.shape

(3000, 192, 3)

In [ ]:
# To address error that occurs when training with the data. Doesn't support format float 64
train_data_midi_temp = np.float32(train_data_midi_temp)

In [ ]:
training_data_midi = np.float32(training_data_midi)

In [ ]:
# Cutting our length of the matrix to properly fit size of the output
train_dataset_midi = tf.data.Dataset.from_tensor_slices(training_data_midi[:,:192,:]) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_dataset_midi = tf.data.Dataset.from_tensor_slices(train_data_midi_temp) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_dataset_midi

<BatchDataset shapes: (None, 192, 3), types: tf.float32>

In [ ]:
def build_generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*256, activation="relu",input_dim=SEED_SIZE))
    model.add(Reshape((4,256)))

    model.add(UpSampling1D())
    model.add(Conv1D(256,kernel_size=2,padding="same"))
    model.add(BatchNormalization(momentum=0.5))
    model.add(Activation("relu"))

    model.add(UpSampling1D())
    model.add(Conv1D(128,kernel_size=2,padding="same"))
    model.add(BatchNormalization(momentum=0.5))
    model.add(Activation("relu"))

    # Output resolution, additional upsampling
    model.add(UpSampling1D())
    model.add(Conv1D(64,kernel_size=2,padding="same"))
    model.add(BatchNormalization(momentum=0.5))
    model.add(Activation("relu"))

    model.add(UpSampling1D())
    model.add(Conv1D(32,kernel_size=2,padding="same"))
    model.add(BatchNormalization(momentum=0.5))
    model.add(Activation("relu"))

    model.add(UpSampling1D(size=3))
    model.add(Conv1D(16,kernel_size=2,padding="same"))
    model.add(BatchNormalization(momentum=0.5))
    model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv1D(3,kernel_size=2,padding="same"))
    model.add(Activation("tanh"))
    
    return model

In [ ]:
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv1D(16, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding1D(padding=((0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding1D(padding=((0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(256, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(512, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv1D(16, kernel_size=2, strides=1, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding1D(padding=((0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(64, kernel_size=2, strides=1, padding="same"))
    model.add(ZeroPadding1D(padding=((0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(256, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(512, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
image_shape = (192,3)

generator = build_generator(SEED_SIZE, 3)
discriminator = build_discriminator(image_shape)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1.0e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(7.5e-5,0.5)

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(seed, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    

        gradients_of_generator = gen_tape.gradient(\
            gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(\
            disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(
            gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(
            gradients_of_discriminator, 
            discriminator.trainable_variables))
    return gen_loss,disc_loss

In [ ]:
def train(dataset, epochs):
    #fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
    #                                   SEED_SIZE))
    start = time.time()

    for epoch in range(epochs):
        epoch_start = time.time()

        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            t = train_step(image_batch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)

        epoch_elapsed = time.time()-epoch_start
        print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'\
           ' {hms_string(epoch_elapsed)}')
        #save_images(epoch,fixed_seed)

    elapsed = time.time()-start
    print (f'Training time: {hms_string(elapsed)}')

In [ ]:
train(train_dataset_midi, 50)

Epoch 1, gen loss=3.731968402862549,disc loss=0.2561679184436798, {hms_string(epoch_elapsed)}
Epoch 2, gen loss=3.63586163520813,disc loss=0.3966277539730072, {hms_string(epoch_elapsed)}
Epoch 3, gen loss=1.702225923538208,disc loss=1.0157688856124878, {hms_string(epoch_elapsed)}
Epoch 4, gen loss=3.0801422595977783,disc loss=0.3693147599697113, {hms_string(epoch_elapsed)}
Epoch 5, gen loss=3.473533868789673,disc loss=0.2360735386610031, {hms_string(epoch_elapsed)}
Epoch 6, gen loss=3.6515514850616455,disc loss=0.2515440881252289, {hms_string(epoch_elapsed)}
Epoch 7, gen loss=3.4490461349487305,disc loss=0.3931594491004944, {hms_string(epoch_elapsed)}
Epoch 8, gen loss=3.050719976425171,disc loss=0.4431723952293396, {hms_string(epoch_elapsed)}
Epoch 9, gen loss=3.0776309967041016,disc loss=0.42446452379226685, {hms_string(epoch_elapsed)}
Epoch 10, gen loss=2.9805874824523926,disc loss=0.47683197259902954, {hms_string(epoch_elapsed)}
Epoch 11, gen loss=2.921447515487671,disc loss=0.4917

In [ ]:
generator.save(os.path.join(DATA_PATH,"song_generator.h5"))

In [ ]:
noise = tf.random.normal([1, SEED_SIZE])
generated_song = generator(noise, training=False)
decision = discriminator(generated_song)
print(decision)

tf.Tensor([[0.06564915]], shape=(1, 1), dtype=float32)


In [ ]:
def recreate_midi(df_first_notes, speed=20000):
    # function to take a dataframe created by something like parse_notes() or a gan and return a midi
    
    # Can start by reverse scaling the note:
    df_reversed = df_first_notes.copy()
    df_reversed['note'] = round(df_reversed['note'] * 88 + 20)  # might want to have something more special than round()
    df_reversed.note = df_reversed.note.astype(int)
    df_reversed['velocity'] = 60  # create a uniform middling velocity

    # recreate the absolute time index and drop time_since_last (we'll recreate it with the stop signals)
    df_reversed['time_index'] = df_reversed.time_since_last.cumsum()
    df_reversed = df_reversed.drop(columns = 'time_since_last')

    # create a stop signal for each note at the appropriate time_index:
    for i in range(len(df_reversed)):
        stop_note = pd.DataFrame([[df_reversed.note[i], 0, 0, df_reversed.duration[i] + df_reversed.time_index[i]]],
                                 columns=['note', 'duration', 'velocity', 'time_index'])
        df_reversed = df_reversed.append(stop_note, ignore_index=True)
    df_reversed = df_reversed.sort_values('time_index').reset_index(drop=True)

    # recreate time_since last with the stop note signals
    df_reversed['time'] = [0] + [df_reversed.time_index[i+1] - df_reversed.time_index[i] 
                                 for i in range(len(df_reversed)-1)]
    # and now we don't need duration or time_index so can drop those
    df_reversed = df_reversed.drop(columns = {'time_index','duration'})

    # finally, we need to scale the time since last note appropriately:
    df_reversed['time'] = round(df_reversed['time'] * speed)
    df_reversed.time = df_reversed.time.astype(int)

    # finally, recreate the midi and return
    mid_remade = MidiFile()
    track = MidiTrack()
    mid_remade.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    for i in range(len(df_reversed)):
        track.append(Message('note_on', note=df_reversed.note[i], velocity=df_reversed.velocity[i], time=df_reversed.time[i]))

    return mid_remade

In [ ]:
midi_dat = generated_song.numpy()
midi_dat

array([[[ 3.56886059e-01,  5.03877318e-03,  1.92426692e-03],
        [ 2.18896002e-01,  8.60210881e-03,  9.39178164e-04],
        [ 2.52768606e-01,  6.84656296e-03,  1.15764630e-03],
        [ 2.57610410e-01,  5.23065170e-03,  1.47788657e-03],
        [ 2.72206236e-02,  8.32996704e-03,  4.90020448e-03],
        [ 2.39114657e-01,  3.43915587e-03,  1.27979869e-03],
        [ 2.53532767e-01,  3.59195424e-03,  7.88656995e-04],
        [ 8.10580775e-02,  6.83449907e-03,  2.81386590e-03],
        [ 2.43380338e-01, -5.81752334e-04,  3.20194056e-03],
        [ 2.94054031e-01,  2.29520001e-03,  2.43110419e-03],
        [ 5.08092582e-01,  6.55644573e-04,  3.53159606e-02],
        [ 4.11900133e-01,  4.86456789e-02,  1.95494201e-02],
        [ 4.10407245e-01,  3.42631601e-02,  2.29802541e-03],
        [ 3.46860528e-01,  2.12745927e-02, -4.45137126e-03],
        [ 4.55896050e-01,  1.80639680e-02,  1.36632810e-03],
        [ 4.72727269e-01,  1.11953001e-02,  2.92960089e-03],
        [ 4.05573398e-01

In [ ]:
# To make the inference output into a mid file
midi_df = pd.DataFrame(midi_dat[0],columns=["note", "duration",'time_since_last'])
mid_made = recreate_midi(midi_df)
mid_made.save('mid_test_7.mid')